In [ ]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))

import pandas as pd
from pycaret.classification import ClassificationExperiment
import src.utils as utils
import src.calcEMA as calc_utils
import src.myenv as myenv
from datetime import datetime

from itertools import combinations
import plotly.express as px

In [ ]:
# Variables
# ETCUSDT BTCUSDT
# symbol = 'ETHUSDT'
symbol = 'BTCUSDT'
interval = '1h'
# lightgbm  xgboost lr et rf
estimator = 'knn'
_compare_models = False

start_train_date = '2010-01-01'  # train < and test >=
start_test_date = '2023-01-01'  # train < and test >=

stop_loss = 1.0
regression_times = 0  # 24 * 30 * 2  # horas
times_regression_PnL = 24
# numeric_features=['open', 'high', 'low', 'volume', 'close', 'rsi']
numeric_features = ['open', 'high', 'low', 'volume', 'close', ]
normalize = True
use_gpu = False
tune_model = False
apply_combination_features = True

### Metadata

<code>
Field Name - Description</br>
open_time - Kline Open time in unix time format</br>
open - Open Price</br>
high - High Price</br>
low	- Low Price</br>
close	- Close Price</br>
volume - Volume</br>
close_time - Kline Close time in unix time format</br>
quote_volume - Quote Asset Volume</br>
count	- Number of Trades</br>
taker_buy_volume - Taker buy base asset volume during this period</br>
taker_buy_quote_volume - Taker buy quote asset volume during this period</br>
ignore - Ignore</br>
</code>

In [ ]:
cols = myenv.all_klines_cols.copy()
cols.remove('ignore')
data = utils.get_data(symbol=symbol, save_database=False, interval=interval, tail=-1, columns=cols, parse_dates=False)
data = utils.parse_type_fields(data, parse_dates=True)
data = utils.adjust_index(data)
data.info()
data

In [ ]:
data = calc_utils.calc_RSI(data)
data.info()
data

In [ ]:
data = calc_utils.calc_ema_periods(data, periods_of_time=[times_regression_PnL, 200])
data.info()
data

In [ ]:
data = utils.regression_PnL(
    data=data,
    label=myenv.label,
    diff_percent=float(stop_loss),
    max_regression_profit_and_loss=int(times_regression_PnL),
    drop_na=True,
    drop_calc_cols=True,
    strategy=None)
data.info()
data

In [ ]:
perc_data_label = data[[myenv.label, 'open_time']].groupby(myenv.label).count()
perc_data_label['perc'] = perc_data_label['open_time'] / len(data)
perc_data_label

In [ ]:
train_data = data[(data['open_time'] >= start_train_date) & (data['open_time'] < start_test_date)]
train_data = train_data.sort_values(myenv.date_features)
train_data

In [ ]:
test_data = data[data['open_time'] >= start_test_date]
test_data = test_data.sort_values(myenv.date_features)
test_data

In [ ]:
# BTCUSDT 1h best params: close,volume,quote_asset_volume,number_of_trades,rsi
# numeric_features = 'volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p'.split(',')
text_numeric_features = 'close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,rsi,ema_24p,ema_200p'
numeric_features = text_numeric_features.split(',')
print(f'Numeric Features: {numeric_features} - size: {len(numeric_features)}\n')

combination_numeric_features = []
if apply_combination_features:
	for size in range(1, len(numeric_features) + 1): 
		comb = map(list, combinations(numeric_features, size))	
		for c in comb:
			res = ''
			for j in c:
				res += f'{j},'
			combination_numeric_features.append(res[0:len(res) - 1])
else:
	combination_numeric_features = [text_numeric_features]

print(f'Combination Numeric Features: \n{combination_numeric_features}')

In [11]:
from datetime import datetime
#fix_imbalance_method: condensednearestneighbour, editednearestneighborus, repeatededitednearestneighbours, allknn, instancehardnessthreshold, nearmiss, neighbourhoodcleaningrule, onesidedselection, randomundersampler, tomeklinks, randomoversampler, smote, smotenc, smoten, adasyn, borderlinesmote, kmeanssmote, svmsmote, smoteenn, smotetomek.
#'condensednearestneighbour,editednearestneighborus,repeatededitednearestneighbours,allknn,instancehardnessthreshold,nearmiss,neighbourhoodcleaningrule,onesidedselection,randomundersampler,tomeklinks,randomoversampler,smote,smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
#'smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
imbalance_list = 'editednearestneighborus,repeatededitednearestneighbours,allknn,instancehardnessthreshold,nearmiss,neighbourhoodcleaningrule,onesidedselection,randomundersampler,tomeklinks,randomoversampler,smote,smotenc,smoten,adasyn,borderlinesmote,kmeanssmote,svmsmote,smoteenn,smotetomek'.split(',')
simulation_results_filename = f'./resultado_simulacao_{symbol}_{interval}.csv'

df_resultado_simulacao = pd.read_csv(simulation_results_filename, sep=';')
for aux_numeric_features in combination_numeric_features:
  experiement = ClassificationExperiment()
  features = []
  features += ['open_time', myenv.label]
  features += aux_numeric_features.split(',')
  print(f'features: {features}')
  for fix_imbalance_method in imbalance_list:
    try:
      print(f'fix_imbalance_method: {fix_imbalance_method}')
      setup = experiement.setup(
          data=train_data[features].copy(),
          train_size=myenv.train_size,
          target=myenv.label,
          numeric_features=aux_numeric_features.split(','),
          date_features=['open_time'],
          create_date_columns=["hour", "day", "month"],
          data_split_shuffle=False, 
          data_split_stratify=False,
          fix_imbalance=True,
          fix_imbalance_method=fix_imbalance_method,
          remove_outliers=True,
          fold_strategy='timeseries',
          fold=3,
          session_id=123,
          normalize=normalize,
          use_gpu=use_gpu,
          verbose=True,
          n_jobs=20,
          log_experiment=False,
          )

      if _compare_models:
        best = setup.compare_models()
        estimator = setup.pull().index[0]
        print('Estimator: ' + estimator)
      else:
        best = setup.create_model(estimator)

      if tune_model:
        best = setup.tune_model(best)

      # predict on test set
      # holdout_pred = setup.predict_model(best)
      # print('Holdout Score:', holdout_pred['prediction_score'].mean())
      # print('Holdout Score Group:\n', holdout_pred[[myenv.label, 'prediction_score']].groupby(myenv.label).mean())

      predict = setup.predict_model(best, data=test_data.drop(columns=[myenv.label]))
      predict[myenv.label] = test_data[myenv.label]
      predict['_score'] = predict['prediction_label'] == predict[myenv.label]
      # print('Predict Score Mean:', predict['_score'].mean())
      # print('Predict Score Mean Group:\n', predict[[myenv.label, '_score']].groupby(myenv.label).mean())

      final_model = setup.finalize_model(best)

      ajusted_test_data = test_data.drop(myenv.label, axis=1)
      df_final_predict, res_score = utils.validate_score_test_data(
          setup,
          final_model,
          myenv.label,
          test_data,
          ajusted_test_data)

      #df_final_predict.info()
      # print('Final Score Mean:', res_score.mean().values[0])
      # print('Final Score Group:\n', res_score)

      start_test_date = df_final_predict['open_time'].min()
      end_test_date = df_final_predict['open_time'].max()

      # print('Simule Trading:')
      # print(f'Min Data: {start_test_date}')
      # print(f'Max Data: {end_test_date}')
      saldo_inicial = 100.0
      saldo_final = utils.simule_trading_crypto2(df_final_predict, start_test_date, end_test_date, saldo_inicial, stop_loss)
      print(f'>>>> Saldo Final: {saldo_final} - features: {features}\n\n')

      result_simulado = {}
      result_simulado['date'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
      result_simulado['symbol'] = symbol
      result_simulado['interval'] = interval
      result_simulado['estimator']  = estimator
      result_simulado['fix_imbalance_method'] = fix_imbalance_method
      result_simulado['stop_loss'] = stop_loss
      result_simulado['times_regression_profit_and_loss'] = times_regression_PnL
      result_simulado['features'] = features
      result_simulado['final_value'] = saldo_final

      if res_score is not None:
        result_simulado['score'] = ''
        for i in range(0, len(res_score.index.values)):
          result_simulado['score'] += f'[{res_score.index.values[i]}={res_score["_score"].values[i]:.2f}]'
      
      df_resultado_simulacao = pd.concat([df_resultado_simulacao, pd.DataFrame([result_simulado])], ignore_index=True)
      df_resultado_simulacao.sort_values('final_value', inplace=True)

      df_resultado_simulacao.to_csv(simulation_results_filename, sep=';', index=False)
    except Exception as e:
      print(e)
      continue
      





,Description,Value
0,Session id,123
1,Target,status
2,Target type,Multiclass
3,Target mapping,"CAI_1.0: 0, ESTAVEL: 1, SOBE_1.0: 2"
4,Original data shape,"(37013, 5)"
5,Transformed data shape,"(20754, 7)"
6,Transformed train set shape,"(9650, 7)"
7,Transformed test set shape,"(11104, 7)"
8,Numeric features,3
9,Date features,1


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.4229,0.5129,0.4229,0.4249,0.4218,0.0187,0.0188
1,0.3773,0.4656,0.3773,0.3674,0.3702,-0.0484,-0.0488
2,0.4831,0.4970,0.4831,0.4939,0.4680,0.0000,0.0000
Mean,0.4278,0.4918,0.4278,0.4287,0.4200,-0.0099,-0.0100
Std,0.0433,0.0196,0.0433,0.0517,0.0400,0.0283,0.0285


>>>> Saldo Final: 146.2940603594859 - features: ['open_time', 'status', 'close', 'number_of_trades', 'taker_buy_quote_asset_volume']


fix_imbalance_method: nearmiss


,Description,Value
0,Session id,123
1,Target,status
2,Target type,Multiclass
3,Target mapping,"CAI_1.0: 0, ESTAVEL: 1, SOBE_1.0: 2"
4,Original data shape,"(37013, 5)"
5,Transformed data shape,"(20473, 7)"
6,Transformed train set shape,"(9369, 7)"
7,Transformed test set shape,"(11104, 7)"
8,Numeric features,3
9,Date features,1


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.2415,0.4710,0.2415,0.3840,0.2729,-0.0381,-0.0471
1,0.1612,0.4454,0.1612,0.3289,0.1518,-0.0585,-0.0956
2,0.0062,0.5000,0.0062,0.0000,0.0001,0.0000,0.0000
Mean,0.1363,0.4721,0.1363,0.2377,0.1416,-0.0322,-0.0476
Std,0.0977,0.0223,0.0977,0.1695,0.1116,0.0243,0.0390


...

,Description,Value
0,Session id,123
1,Target,status
2,Target type,Multiclass
3,Target mapping,"CAI_1.0: 0, ESTAVEL: 1, SOBE_1.0: 2"
4,Original data shape,"(37013, 5)"
5,Transformed data shape,"(30031, 7)"
6,Transformed train set shape,"(18927, 7)"
7,Transformed test set shape,"(11104, 7)"
8,Numeric features,3
9,Date features,1


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.4067,0.4955,0.4067,0.4159,0.3890,0.0024,0.0025
1,0.3900,0.4697,0.3900,0.3740,0.3655,-0.0495,-0.0518
2,0.5158,0.4973,0.5158,0.2661,0.3511,0.0000,0.0000
Mean,0.4375,0.4875,0.4375,0.3520,0.3685,-0.0157,-0.0164
Std,0.0558,0.0126,0.0558,0.0631,0.0156,0.0239,0.0250


>>>> Saldo Final: 344.36853114595453 - features: ['open_time', 'status', 'close', 'number_of_trades', 'ema_200p']


fix_imbalance_method: randomundersampler


,Description,Value
0,Session id,123
1,Target,status
2,Target type,Multiclass
3,Target mapping,"CAI_1.0: 0, ESTAVEL: 1, SOBE_1.0: 2"
4,Original data shape,"(37013, 5)"
5,Transformed data shape,"(20443, 7)"
6,Transformed train set shape,"(9339, 7)"
7,Transformed test set shape,"(11104, 7)"
8,Numeric features,3
9,Date features,1


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.4138,0.5000,0.4138,0.4166,0.4113,-0.0006,-0.0006
1,0.3897,0.4818,0.3897,0.3788,0.3665,-0.0319,-0.0339
2,0.4575,0.5032,0.4575,0.5029,0.4783,0.0137,0.0139
Mean,0.4203,0.4950,0.4203,0.4328,0.4187,-0.0063,-0.0068
Std,0.0281,0.0094,0.0281,0.0519,0.0459,0.0190,0.0200
